In [ ]:
! git clone https://github.com/jimmyjbling/SGC-DEL-ML-WDR91.git
! git clone https://github.com/NikSchap-2107/Target2035_Aircheck_Utils.git

In [ ]:
import sys
sys.path.append("/content/Target2035_Aircheck_Utils/ReadParquetFiles")
sys.path.append("/content/SGC-DEL-ML-WDR91/scripts")
from read_parquet_utils import read_parquet_file
from sklearn.model_selection import train_test_split
import numpy as np

data_file = "/content/Target2035_Aircheck_Utils/Data/WDR91.parquet"
df = read_parquet_file(data_file, columns = ['ECFP4', 'LABEL'])